In [4]:
!pip install numpy pandas scikit-learn nltk keras tensorflow flask



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [5]:
!pip install --upgrade tensorflow
!pip install --upgrade keras



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [6]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk

from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.base import BaseEstimator, ClassifierMixin

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Embedding, Dense, LSTM, Input




nltk.download('stopwords')
from nltk.corpus import stopwords



[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/luisdavid/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [11]:
!pip install kaggle



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [7]:
!pwd

/Users/luisdavid/Desktop/9A/Mario/U3


In [8]:
os.environ['KAGGLE_CONFIG_DIR'] = '/content'

In [13]:
!kaggle datasets download -d kritanjalijain/amazon-reviews

Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.12/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/kaggle/__init__.py", line 3, in <module>
    from kaggle.api.kaggle_api_extended import KaggleApi
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/kaggle/api/__init__.py", line 6, in <module>
    from kaggle.api.kaggle_api_extended import KaggleApi
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/kaggle/api/kaggle_api_extended.py", line 258, in <module>
    class KaggleApi(KaggleApi):
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/kaggle/api/kaggle_api_extended.py", line 289, in KaggleApi
    os.makedirs(config_dir)
  File "<frozen os>", line 225, in makedirs
OSError: [Errno 30] Read-only file system: '/content'


In [14]:
!unzip amazon-reviews.zip -d amazondata


Archive:  amazon-reviews.zip
  inflating: amazondata/amazon_review_polarity_csv.tgz  
  inflating: amazondata/test.csv     
  inflating: amazondata/train.csv    


In [ ]:
# Parte 1: Preparación y Preprocesamiento

import pandas as pd
import numpy as np
import re
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Función para cargar y formatear datos
def load_data(file_path):
    # Cargar datos sin encabezados
    data = pd.read_csv(file_path, header=None)
    # Nombrar columnas basándonos en el formato proporcionado
    data.columns = ["label", "title", "review_text"]
    return data

# Función para limpiar texto
def clean_text(text):
    # Quitar caracteres especiales, puntuaciones y números
    text = re.sub(f"[{re.escape(string.punctuation)}]", "", text)
    text = re.sub(r'\d+', '', text)
    text = text.lower()  # Convertir a minúsculas
    text = re.sub(r'\s+', ' ', text).strip()  # Eliminar espacios adicionales
    return text

# Preprocesamiento general
def preprocess_data(data):
    # Limpiar la columna de texto
    data["cleaned_review_text"] = data["review_text"].apply(clean_text)
    # Convertir etiquetas a 0 (negativo) y 1 (positivo)
    data["label"] = data["label"].apply(lambda x: 0 if x == 1 else 1)
    return data

# Vectorización TFIDF
def prepare_tfidf_vectorizer(X_train, X_test):
    vectorizer = TfidfVectorizer(max_features=50000, stop_words='english')
    X_train_tfidf = vectorizer.fit_transform(X_train)
    X_test_tfidf = vectorizer.transform(X_test)
    return vectorizer, X_train_tfidf, X_test_tfidf

# Preparación para DNN
def prepare_dnn_data(X_train, X_test, max_vocab_size=50000, max_sequence_length=100):
    tokenizer = Tokenizer(num_words=max_vocab_size)
    tokenizer.fit_on_texts(X_train)
    X_train_seq = pad_sequences(tokenizer.texts_to_sequences(X_train), maxlen=max_sequence_length)
    X_test_seq = pad_sequences(tokenizer.texts_to_sequences(X_test), maxlen=max_sequence_length)
    return tokenizer, X_train_seq, X_test_seq

# Rutas de los datos
train_path = "amazondata/train.csv"
test_path = "amazondata/test.csv"

# Cargar y preprocesar datos
train_data = load_data(train_path)
test_data = load_data(test_path)

train_data = preprocess_data(train_data)
test_data = preprocess_data(test_data)

# División en características y etiquetas
X_train, y_train = train_data["cleaned_review_text"], train_data["label"]
X_test, y_test = test_data["cleaned_review_text"], test_data["label"]

# Preparar datos para TFIDF
vectorizer, X_train_tfidf, X_test_tfidf = prepare_tfidf_vectorizer(X_train, X_test)

# Preparar datos para DNN
tokenizer, X_train_seq, X_test_seq = prepare_dnn_data(X_train, X_test)

print("Datos listos para los modelos:")
print(f"TFIDF - X_train: {X_train_tfidf.shape}, X_test: {X_test_tfidf.shape}")
print(f"DNN - X_train_seq: {X_train_seq.shape}, X_test_seq: {X_test_seq.shape}")




Datos listos para los modelos:
TFIDF - X_train: (3600000, 50000), X_test: (400000, 50000)
DNN - X_train_seq: (3600000, 100), X_test_seq: (400000, 100)


Clasificador TFIDF + Regresión Logística
Incluye:

Preprocesamiento.
Implementación como clase.
Validación cruzada (K-folds).
Generación del informe de métricas.

In [28]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import classification_report

class LogisticTfidfClassifier:
    def __init__(self, max_features=50000):
        self.vectorizer = TfidfVectorizer(max_features=max_features, stop_words='english')
        self.model = LogisticRegression(max_iter=1000)

    def train(self, X_train, y_train):
        X_train_tfidf = self.vectorizer.fit_transform(X_train)
        self.model.fit(X_train_tfidf, y_train)

    def predict(self, X_test):
        X_test_tfidf = self.vectorizer.transform(X_test)
        return self.model.predict(X_test_tfidf)

    def cross_validate(self, X, y, cv=5):
        X_tfidf = self.vectorizer.fit_transform(X)
        scores = cross_val_score(self.model, X_tfidf, y, cv=cv, scoring='accuracy')
        return scores.mean()

    def evaluate(self, X_test, y_test):
        y_pred = self.predict(X_test)
        report = classification_report(y_test, y_pred, target_names=["Negative", "Positive"])
        return report

# Entrenamiento y evaluación del clasificador
logistic_tfidf = LogisticTfidfClassifier()
logistic_tfidf.train(X_train, y_train)

# Validación cruzada
cv_score = logistic_tfidf.cross_validate(X_train, y_train)
print(f"Cross-validation accuracy: {cv_score:.4f}")

# Informe de clasificación
logistic_report = logistic_tfidf.evaluate(X_test, y_test)
print("Logistic Regression + TFIDF Classification Report:")
print(logistic_report)


Cross-validation accuracy: 0.8717
Logistic Regression + TFIDF Classification Report:
              precision    recall  f1-score   support

    Negative       0.88      0.87      0.87    200000
    Positive       0.87      0.88      0.87    200000

    accuracy                           0.87    400000
   macro avg       0.87      0.87      0.87    400000
weighted avg       0.87      0.87      0.87    400000



2. Red Neuronal Profunda (DNN)
Incluye:

Arquitectura como clase.
Entrenamiento y evaluación.
Generación del informe de métricas.

In [30]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from sklearn.metrics import classification_report
import numpy as np

class DNNClassifier:
    def __init__(self, vocab_size=50000, sequence_length=100, embedding_dim=128):
        self.model = Sequential([
            Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=sequence_length),
            LSTM(128, return_sequences=True),
            Dropout(0.5),
            LSTM(64),
            Dense(32, activation='relu'),
            Dense(1, activation='sigmoid')
        ])
        self.model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    def train(self, X_train, y_train, epochs=5, batch_size=32, validation_data=None):
        self.model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=validation_data)

    def predict(self, X_test):
        predictions = self.model.predict(X_test)
        return (predictions > 0.5).astype(int).flatten()

    def evaluate(self, X_test, y_test):
        y_pred = self.predict(X_test)
        report = classification_report(y_test, y_pred, target_names=["Negative", "Positive"])
        return report

# Entrenamiento y evaluación del clasificador
dnn_classifier = DNNClassifier(vocab_size=50000, sequence_length=100)
dnn_classifier.train(X_train_seq, y_train, epochs=3, batch_size=64, validation_data=(X_test_seq, y_test))

# Informe de clasificación
dnn_report = dnn_classifier.evaluate(X_test_seq, y_test)
print("DNN Classification Report:")
print(dnn_report)


Epoch 1/3
56250/56250 ━━━━━━━━━━━━━━━━━━━━ 7348s 131ms/step - accuracy: 0.9051 - loss: 0.2323 - val_accuracy: 0.9348 - val_loss: 0.1700
Epoch 2/3
56250/56250 ━━━━━━━━━━━━━━━━━━━━ 7234s 129ms/step - accuracy: 0.9428 - loss: 0.1522 - val_accuracy: 0.9381 - val_loss: 0.1624
Epoch 3/3
56250/56250 ━━━━━━━━━━━━━━━━━━━━ 7410s 132ms/step - accuracy: 0.9511 - loss: 0.1324 - val_accuracy: 0.9377 - val_loss: 0.1634
12500/12500 ━━━━━━━━━━━━━━━━━━━━ 294s 23ms/step
DNN Classification Report:
              precision    recall  f1-score   support

    Negative       0.93      0.94      0.94    200000
    Positive       0.94      0.93      0.94    200000

    accuracy                           0.94    400000
   macro avg       0.94      0.94      0.94    400000
weighted avg       0.94      0.94      0.94    400000



In [54]:
from sklearn.model_selection import KFold
from tensorflow.keras.callbacks import EarlyStopping

# Validación cruzada para DNN
def cross_validate_dnn(X, y, vocab_size=50000, sequence_length=100, embedding_dim=128, epochs=3, batch_size=64, k=5):
    kfold = KFold(n_splits=k, shuffle=True, random_state=42)
    fold_accuracies = []

    for train_index, val_index in kfold.split(X):
        X_train, X_val = X[train_index], X[val_index]
        y_train, y_val = y[train_index], y[val_index]

        # Crear modelo DNN para cada pliegue
        model = Sequential([
            Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=sequence_length),
            LSTM(128, return_sequences=True),
            Dropout(0.5),
            LSTM(64),
            Dense(32, activation='relu'),
            Dense(1, activation='sigmoid')
        ])
        model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

        # Entrenar modelo
        early_stopping = EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)
        model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_val, y_val), verbose=0, callbacks=[early_stopping])

        # Evaluar modelo
        _, accuracy = model.evaluate(X_val, y_val, verbose=0)
        fold_accuracies.append(accuracy)

    # Retornar la precisión promedio
    return np.mean(fold_accuracies)

# Aplicar validación cruzada en DNN
dnn_cv_accuracy = cross_validate_dnn(
    X_train_seq, y_train.values,
    vocab_size=50000, sequence_length=100, embedding_dim=128, epochs=3, batch_size=64, k=5
)
print(f"Cross-validation accuracy (DNN): {dnn_cv_accuracy:.4f}")


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


KeyboardInterrupt: 

5. Generador de Polaridad Invertida
Preparación de Datos
Convertimos reseñas positivas en negativas (y viceversa) creando pares de datos.

In [35]:
# Verificar el tipo de datos en generator_data
print(type(generator_data[0][0]), type(generator_data[0][1]))



<class 'str'> <class 'int'>


In [36]:
# Asegurar que todos los textos sean cadenas
generator_data = [(str(pair[0]), str(pair[1])) for pair in generator_data]

# Verificar después de la conversión
print(type(generator_data[0][0]), type(generator_data[0][1]))



<class 'str'> <class 'str'>


In [38]:
# Convertir a índices en lugar de one-hot
decoder_target_data = pad_sequences(tokenizer.texts_to_sequences([pair[1] + "\n" for pair in generator_data]), maxlen=100)


In [40]:
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense
from tensorflow.keras.models import Model

# Parámetros
vocab_size = 50000  # Tamaño del vocabulario
embedding_dim = 256

# Entrada del codificador
encoder_inputs = Input(shape=(None,))
encoder_embedding = Embedding(vocab_size, embedding_dim)(encoder_inputs)
encoder_lstm = LSTM(256, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_embedding)
encoder_states = [state_h, state_c]

# Entrada del decodificador
decoder_inputs = Input(shape=(None,))
decoder_embedding = Embedding(vocab_size, embedding_dim)(decoder_inputs)
decoder_lstm = LSTM(256, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)
decoder_dense = Dense(vocab_size, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Modelo de entrenamiento
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')

print(model.summary())


Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_3       │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_4       │ (None, None)      │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_2         │ (None, None, 256) │ 12,800,000 │ input_layer_3[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_3         │ (None, None, 256) │ 12,800,000 │ input_layer_4[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_4 (LSTM)       │ [(None, 256),     │    525,312 │ embedding_2[0][0] │
│                     │ (None, 256),      │            │                   │
│                     │ (None, 256)]      │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_5 (LSTM)       │ [(None, None,     │    525,312 │ embedding_3[0][0… │
│                     │ 256), (None,      │            │ lstm_4[0][1],     │
│                     │ 256), (None,      │            │ lstm_4[0][2]      │
│                     │ 256)]             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_4 (Dense)     │ (None, None,      │ 12,850,000 │ lstm_5[0][0]      │
│                     │ 50000)            │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 39,500,624 (150.68 MB)

 Trainable params: 39,500,624 (150.68 MB)

 Non-trainable params: 0 (0.00 B)

None


encorer

In [48]:
# Implementing the full encoder-decoder model construction, training, and inference.

import numpy as np
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Mock generator data for testing purposes
generator_data = [
    ("This product is amazing", "This product is terrible"),
    ("I loved this book", "I hated this book"),
    ("The movie was fantastic", "The movie was awful"),
]

# Parameters
vocab_size = 50000  # Reduced vocabulary for testing
embedding_dim = 256
max_seq_length = 10

# Tokenizer setup
tokenizer = Tokenizer(num_words=vocab_size, filters='', oov_token='<OOV>')
tokenizer.fit_on_texts([pair[0] for pair in generator_data] + [pair[1] for pair in generator_data])

# Prepare data
encoder_input_data = pad_sequences(
    tokenizer.texts_to_sequences([pair[0] for pair in generator_data]), maxlen=max_seq_length
)
decoder_input_data = pad_sequences(
    tokenizer.texts_to_sequences(["\t" + pair[1] for pair in generator_data]), maxlen=max_seq_length
)
decoder_target_data = pad_sequences(
    tokenizer.texts_to_sequences([pair[1] + "\n" for pair in generator_data]), maxlen=max_seq_length
)

# Encoder
encoder_inputs = Input(shape=(None,))
encoder_embedding = Embedding(vocab_size, embedding_dim)(encoder_inputs)
encoder_lstm = LSTM(256, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_embedding)
encoder_states = [state_h, state_c]

# Decoder
decoder_inputs = Input(shape=(None,))
decoder_embedding = Embedding(vocab_size, embedding_dim)(decoder_inputs)
decoder_lstm = LSTM(256, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)
decoder_dense = Dense(vocab_size, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Training model
training_model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
training_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')

# Inference models
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(256,))
decoder_state_input_c = Input(shape=(256,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

decoder_embedding_inference = Embedding(vocab_size, embedding_dim)(decoder_inputs)
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_embedding_inference, initial_state=decoder_states_inputs
)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states
)

# Training the model
training_model.fit(
    [encoder_input_data, decoder_input_data],
    np.expand_dims(decoder_target_data, -1),
    batch_size=1,
    epochs=10,
    verbose=1
)

# Inference function
def decode_sequence(input_seq):
    states_value = encoder_model.predict(input_seq)
    target_seq = np.zeros((1, 1))
    target_seq[0, 0] = tokenizer.word_index.get('\t', 0)
    stop_condition = False
    decoded_sentence = ""

    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = tokenizer.index_word.get(sampled_token_index, '')
        decoded_sentence += sampled_char + " "

        if sampled_char == "\n" or len(decoded_sentence) > max_seq_length * 2:
            stop_condition = True

        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = sampled_token_index
        states_value = [h, c]

    return decoded_sentence.strip()

# Test inference
example_seq = pad_sequences(tokenizer.texts_to_sequences(["This product is amazing"]), maxlen=max_seq_length)
generated_review = decode_sequence(example_seq)
generated_review


Epoch 1/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 83ms/step - loss: 10.8137
Epoch 2/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step - loss: 10.7570
Epoch 3/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step - loss: 10.4417
Epoch 4/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step - loss: 9.3510
Epoch 5/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step - loss: 7.9682
Epoch 6/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 82ms/step - loss: 6.3006
Epoch 7/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step - loss: 4.5306
Epoch 8/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step - loss: 2.9633
Epoch 9/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step - loss: 2.0105
Epoch 10/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step - loss: 1.6869
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━

''

In [50]:
import pandas as pd

# Ruta a los datos
train_path = "amazondata/train.csv"  # Actualiza con la ruta real
test_path = "amazondata/test.csv"

# Cargar los datos
train_data = pd.read_csv(train_path, header=None)
test_data = pd.read_csv(test_path, header=None)

# Nombrar las columnas
train_data.columns = ["label", "title", "review_text"]
test_data.columns = ["label", "title", "review_text"]


In [51]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import re
import string

# Limpiar texto
def clean_text(text):
    text = re.sub(f"[{re.escape(string.punctuation)}]", "", text)  # Eliminar puntuación
    text = re.sub(r'\d+', '', text)  # Eliminar números
    text = text.lower()  # Convertir a minúsculas
    return text

# Limpiar y procesar los datos
train_data["review_text"] = train_data["review_text"].apply(clean_text)
test_data["review_text"] = test_data["review_text"].apply(clean_text)

# Tokenización
tokenizer = Tokenizer(num_words=50000)  # Limitar el vocabulario
tokenizer.fit_on_texts(train_data["review_text"])

# Convertir texto a secuencias
max_seq_length = 100  # Longitud máxima de las secuencias
train_sequences = pad_sequences(tokenizer.texts_to_sequences(train_data["review_text"]), maxlen=max_seq_length)
test_sequences = pad_sequences(tokenizer.texts_to_sequences(test_data["review_text"]), maxlen=max_seq_length)

# Convertir etiquetas (1 -> 0, 2 -> 1)
train_labels = train_data["label"].apply(lambda x: 0 if x == 1 else 1).values
test_labels = test_data["label"].apply(lambda x: 0 if x == 1 else 1).values


In [52]:
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense
from tensorflow.keras.models import Model

# Encoder
encoder_inputs = Input(shape=(None,))
encoder_embedding = Embedding(input_dim=50000, output_dim=256)(encoder_inputs)
encoder_lstm = LSTM(256, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_embedding)
encoder_states = [state_h, state_c]

# Decoder
decoder_inputs = Input(shape=(None,))
decoder_embedding = Embedding(input_dim=50000, output_dim=256)(decoder_inputs)
decoder_lstm = LSTM(256, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_embedding, initial_state=encoder_states)
decoder_dense = Dense(50000, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Modelo de entrenamiento
training_model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
training_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')


In [53]:
# Crear datos de entrada y salida para el modelo generativo
decoder_input_data = pad_sequences(tokenizer.texts_to_sequences(["\t" + text for text in train_data["review_text"]]), maxlen=max_seq_length)
decoder_target_data = pad_sequences(tokenizer.texts_to_sequences([text + "\n" for text in train_data["review_text"]]), maxlen=max_seq_length)

# Entrenar
training_model.fit(
    [train_sequences, decoder_input_data],
    np.expand_dims(decoder_target_data, -1),
    batch_size=64,
    epochs=5,
    verbose=1
)


Epoch 1/5
 5592/56250 ━━━━━━━━━━━━━━━━━━━━ 52:08:50 4s/step - loss: 1.9440

KeyboardInterrupt: 

In [ ]:
def decode_sequence(input_seq):
    states_value = encoder_model.predict(input_seq)
    target_seq = np.zeros((1, 1))
    target_seq[0, 0] = tokenizer.word_index.get('\t', 0)

    stop_condition = False
    decoded_sentence = ""

    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = tokenizer.index_word.get(sampled_token_index, '')
        decoded_sentence += sampled_char + " "

        if sampled_char == "\n" or len(decoded_sentence) > max_seq_length * 2:
            stop_condition = True

        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = sampled_token_index
        states_value = [h, c]

    return decoded_sentence.strip()

# Probar con un ejemplo
example_seq = pad_sequences(tokenizer.texts_to_sequences(["This product is amazing"]), maxlen=max_seq_length)
generated_review = decode_sequence(example_seq)
print("Generated Review:", generated_review)
